In [1]:
!pip install pygame

    100% |████████████████████████████████| 11.4MB 4.1MB/s eta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel, DQN, STATE_ENCODING_LENGTH
from trainutil import train_models, test_models, get_epsilon_decay_factor
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
episodes = 200
steps = 1200 

gamma = 0.99

buffer_capacity = 15000
buffer_learn_thresh = 0.5
batch_size = 128

target_update = 10

start_epsilon = 0.99
min_epsilon = 0.01
decay_window = 180
decay = get_epsilon_decay_factor(start_epsilon, min_epsilon, decay_window)


calc_decay_window = episodes - int(buffer_learn_thresh * buffer_capacity / steps)
print("decay_window calc:", calc_decay_window)
print("Learning starts at episode:", int(buffer_learn_thresh * buffer_capacity / steps))
print("Decay rate: ", decay)

decay_window calc: 194
Learning starts at episode: 6
Decay rate:  0.9747946518906935


In [8]:
net = DQN(STATE_ENCODING_LENGTH, 8)
fs.load_net_from_device(net, "drl_fc32", "cpu")

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)

In [4]:
env = GameState()
deep_rl_model = DeepRLModel(
                    epsilon = start_epsilon,
                    min_epsilon = min_epsilon,
                    epsilon_decay = decay,
                    gamma = gamma,
                    buffer_capacity = buffer_capacity,
                    replay_buffer_learn_thresh = buffer_learn_thresh,
                    batch_size = batch_size,
                    lr = 1e-4,
                    model = None)

# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
model_name = "drl_fc32_6"
train_models(env, models, episodes=episodes, steps=steps, target_update=target_update, print_every=1000, model_name=model_name)
# test_models(env, models, steps=2000)


TRAIN MODE
=== Starting Episode 0 ===
----STEP 1000 rewards----
Model 0: -3.464872595550389


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean Episode Loss: nan | Episode Reward: -5.6429 | Mean Reward: -5.6429
=== Starting Episode 1 ===
----STEP 1000 rewards----
Model 0: -9.345746285944614
Mean Episode Loss: nan | Episode Reward: -11.8808 | Mean Reward: -8.7618
=== Starting Episode 2 ===
----STEP 1000 rewards----
Model 0: -4.301523534323579
Mean Episode Loss: nan | Episode Reward: -6.6601 | Mean Reward: -8.0613
=== Starting Episode 3 ===
----STEP 1000 rewards----
Model 0: 1.7827531000930321
Mean Episode Loss: nan | Episode Reward: 2.3681 | Mean Reward: -5.4539
=== Starting Episode 4 ===
----STEP 1000 rewards----
Model 0: 97.5122970706758
Mean Episode Loss: nan | Episode Reward: 100.6416 | Mean Reward: 15.7652
=== Starting Episode 5 ===
----STEP 1000 rewards----
Model 0: -0.41759995898217994
Mean Episode Loss: nan | Episode Reward: -1.4267 | Mean Reward: 12.8999
=== Starting Episode 6 ===
----LEARNING BEGINS----
----STEP 1000 rewards----
Model 0: -2.2656027622261377
Mean Episode Loss: 1.5589 | Episode Reward: -3.8287 | Me

In [ ]:
fs.save_net_to_disk(deep_rl_model.model, model_name)